In [1]:
import torch

import torch.nn as nn

from torchtext import data

In [4]:
seed = 42

torch.manual_seed(seed)

torch.backends.cuda.deterministic=True

Text = data.Field(tokenize='spacy')
Label = data.LabelField(dtype=torch.float)

F:\anaconda\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
F:\anaconda\lib\site-packages\torchtext\data\field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [5]:
from torchtext import datasets

train_data , test_data = datasets.IMDB.splits(Text , Label)

F:\anaconda\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [8]:
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

In [11]:
import random

In [12]:
train_data , valid_data = train_data.split(random_state=random.seed(seed))

In [13]:
print(f'Number of training examples:{len(train_data)}')
print(f'Number of validation exampels: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples:17500
Number of validation exampels: 7500
Number of testing examples: 25000


In [17]:
MAX_VOCAB_SIZE = 25000

Text.build_vocab(train_data , max_size=MAX_VOCAB_SIZE)
Label.build_vocab(train_data)

In [21]:
print(f'Unique tokens in Text vocabulary : {len(Text.vocab)} ')
print(f'Unique tokens in Label vocabulary :{len(Label.vocab)} ')

Unique tokens in Text vocabulary : 25002 
Unique tokens in Label vocabulary :2 


In [22]:
print(Text.vocab.freqs.most_common(20))

[('the', 204028), (',', 193029), ('.', 166896), ('and', 109605), ('a', 109284), ('of', 101127), ('to', 94092), ('is', 76723), ('in', 61384), ('I', 54017), ('it', 53630), ('that', 49362), ('"', 44274), ("'s", 43355), ('this', 42296), ('-', 36980), ('/><br', 35505), ('was', 34930), ('as', 30420), ('with', 30150)]


In [23]:
print(Text.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [26]:
print(Label.vocab.stoi)

defaultdict(None, {'pos': 0, 'neg': 1})


In [30]:
BATCH_SIZE=64


train_iterator , valid_iterator , test_iterator = data.BucketIterator.splits(
(train_data , valid_data , test_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=True

)

# Build the model

In [47]:
class RNN(nn.Module):
    
    def __init__(self , input_dim , embedding_dim , hidden_dim , output_dim):
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim , embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim  , hidden_dim)
        
        self.fc = nn.Linear(hidden_dim , output_dim)
        
    
    def forward(self , text):
        
        #text = [sent len , batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len , batch size ,  emb dim]
        
        output , hidden = self.rnn(embedded)
        
        #output = > [sent len , batch size,  hide dim]
        
        #hidden= > [1 , batch size , hid dim]
        
        assert torch.equal(output[-1,:,:] , hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))     # -> [batch_size , hid dim]

In [76]:
INPUT_DIM = len(Text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM=256
OUTPUT_DIM=1

model = RNN(INPUT_DIM , EMBEDDING_DIM , HIDDEN_DIM , OUTPUT_DIM)

In [79]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')



The model has 2,592,105 trainable parameters


# Train the model

In [80]:
import torch.optim as optim

In [81]:
optimizer = optim.SGD(model.parameters() , lr=1e-3)

In [82]:
criterion = nn.BCEWithLogitsLoss()

In [89]:
def binary_accuracy(preds , y):
    
    #round predictions to the closed integer
    
    rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (rounded_preds==y).float() #convert into float for divison
    
    acc = correct.sum()/len(correct)
    
    return acc

In [84]:
from tqdm import tqdm

In [85]:
def train(model , iterator , optimizer , criterion):
    
    epoch_loss= 0
    
    epoch_acc =0
    
    model.train()
    
    for batch in tqdm(iterator):
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1) #->[batch size ]
        
        loss = criterion(predictions , batch.label)
        
        acc=  binary_accuracy(predictions , batch.label)
        
        loss.backward()
        
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator) , epoch_acc/len(iterator)

In [86]:
def evaluate(model , iterator , criterion):
    
    epoch_loss = 0
    
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions , batch.label)
            
            acc = binary_accuracy(predictions , batch.label)
            
            epoch_loss += loss.item()
            
            epoch_acc += acc.item()
            
    return epoch_loss / len(iterator) , epoch_acc / len(iterator)

In [91]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|                                                                                          | 0/274 [00:00<?, ?it/s]

	Train Loss: 0.694 | Train Acc: 49.78%
	 Val. Loss: 0.694 |  Val. Acc: 49.93%


  0%|                                                                                          | 0/274 [00:00<?, ?it/s]

	Train Loss: 0.693 | Train Acc: 50.40%
	 Val. Loss: 0.694 |  Val. Acc: 50.56%


  0%|                                                                                          | 0/274 [00:00<?, ?it/s]

	Train Loss: 0.693 | Train Acc: 50.28%
	 Val. Loss: 0.693 |  Val. Acc: 50.70%


  0%|▎                                                                                 | 1/274 [00:00<00:30,  8.93it/s]

	Train Loss: 0.693 | Train Acc: 50.80%
	 Val. Loss: 0.693 |  Val. Acc: 51.39%


100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [04:44<00:00,  1.04s/it]


	Train Loss: 0.693 | Train Acc: 50.92%
	 Val. Loss: 0.693 |  Val. Acc: 51.22%


In [92]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.694 | Test Acc: 50.38%


In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()